## 💻 신경망을 이용한 이미지 분류
신경망을 통해 더 정교하고 정확도가 더 높은 머신러닝으로 이미지를 분류한다.

### 신경망 소개
신경망은 기본적으로 퍼셉트론(뉴런)으로 구성되어 있다.  
가장 일반적으로 각 입력에 가중치가 곱해지고 모든 곱의 합이 정규화되어서 출력된다.
기본적으로 퍼셉트론은 입력 세트를 입력 레이어(첫 번째 레이어)에서 고 퍼셉트론의 히든 레이어에서 연산후 다음 레이어로 출력(출력 레이어)을 반환한다. 이 때 입력 레이어에 비해 히든 레이어에 비해 크거나 작지 않아야하며, 출력 레이어의 크기는 레이블의 수에 따라 달라진다. 즉, 출력 레이어는 각 레이블에 해당하는 퍼셉트론을 갖는다.  
#### 네트워크 학습
네트워크 학습 시에는 크게 두 단계로 나눌 수 있다. 첫 번째는 입력을 앞으로 전달하여 입력 값에 퍼셉트론의 가중치를 곱하고 출력이 생성되는 과정, 그리고 출력과 실제 출력의 오차를 이용의 퍼셉트론의 가중치를 수정하는 역전파 과정으로 나뉘어 진다. 이 과정을 여러번 전달하게 되는데, 모든 데이터를 한 번 전달하는 것을 세대(epoch)라고 한다.

### 신경망을 이용한 MNIST 숫자 분류
1. 이미지 픽셀 값을 0,1 혹은 -1,1 사이로 정규화하여 전처리
2. 학습 세트와 테스트 세트로 데이터 세트를 나눈다.
3. 학습 데이터 세트로 학습
4. 테스트 데이터 세트로 네트워크의 성능을 확인한다.

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

In [6]:
print("Getting MNIST Data...")
mnist = fetch_openml('mnist_784', version=1, cache=True)
print("MNIST Data downloaded!")

Getting MNIST Data...
MNIST Data downloaded!


In [7]:
images = mnist.data
labels = mnist.target

In [8]:
images = normalize(images,  norm ="l2") #정규화
images_train, images_test, labels_train, labels_test = train_test_split(images, #학습/테스트 데이터 분할
                                                                        labels, 
                                                                        test_size=0.25, 
                                                                        random_state=17) 

In [16]:
nn = MLPClassifier(hidden_layer_sizes=(100,100,100), #네트워크 크기
                   max_iter = 20,            #학습을 수행할 최대 반복 횟수 설정
                   solver = "sgd", 
                   learning_rate_init=0.001,
                   verbose=True)

In [17]:
print("NN Training started...")
nn.fit(images_train, labels_train)
print("NN Training completed!")

NN Training started...
Iteration 1, loss = 2.30784965
Iteration 2, loss = 2.28616402
Iteration 3, loss = 2.26857418
Iteration 4, loss = 2.24496765
Iteration 5, loss = 2.20764662
Iteration 6, loss = 2.14426503
Iteration 7, loss = 2.03160676
Iteration 8, loss = 1.84484981
Iteration 9, loss = 1.60323469
Iteration 10, loss = 1.37186946
Iteration 11, loss = 1.17817786
Iteration 12, loss = 1.02171217
Iteration 13, loss = 0.90567090
Iteration 14, loss = 0.82203268
Iteration 15, loss = 0.75835112
Iteration 16, loss = 0.70600132
Iteration 17, loss = 0.66164720
Iteration 18, loss = 0.62323073
Iteration 19, loss = 0.58935911
Iteration 20, loss = 0.55952842
NN Training completed!


C:\Users\user\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


In [18]:
print("Network Performance: %f" %nn.score(images_test, labels_test) )

Network Performance: 0.843429


### 합성곱 신경망(Convolutional Neural Network)
기본적으로 합성곱은 커널/필터를 적용시켜 원래 이미지보다 작을 수 있는 새로운 이미지를 만드는 연산이다. CNN에서는 입력 이미지를 시작으로 매번 이미지를 받고 필터를 사용해 합성곱을 적용한다. 따라서 매 합성곱 레이어 이후 출력 이미지는 입력 이미지보다 현저히 작으며 이를 subsampling이라고 한다.   
<img src = "cnn.png" width="800">  
합성곱 사이에서 서브샘플링으로 레이어는 완전히 연결되어 있다 = fully connected layer. FC 레이어에서는 서브샘플링된 이미지를 각각 레이블에 대한 점수 값으로 변환이 되고, 각 출력 값은 각 클래스에 대한 확률 값이다.  
👇🏻  아래 코드는 전통 CNN의 종류인 LeNet을 구현한 코드

In [31]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Activation, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import utils
from sklearn import datasets
from tensorflow.keras import backend as k

In [38]:
k.set_image_data_format("channels_first")

In [39]:
num_classes=10
img_depth=1
img_height=28
img_width=28

In [41]:
model = Sequential()
model.add(Conv2D(20, (5,5), input_shape=(img_depth, img_height, img_width)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

In [42]:
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation("softmax"))

In [45]:
mnist = fetch_openml('mnist_784', version=1, cache=True)

In [46]:
mnist.data = mnist.data.reshape((mnist.data.shape[0], 28,28))
mnist.data = mnist.data[:,np.newaxis,:,:]
mnist.data = mnist.data/255.0

In [48]:
train_data, test_data, train_label, test_label = train_test_split(mnist.data,
                                                                 mnist.target,
                                                                 test_size=0.25)
train_label = utils.to_categorical(train_label, 10)
test_label = utils.to_categorical(test_label, 10)

In [49]:
model.compile(loss="categorical_crossentropy",
              optimizer = SGD(lr=0.001),
              metrics=["accuracy"])
model.fit(train_data,train_label,batch_size=32, epochs=30, verbose=1)

Epoch 1/30
1641/1641 [==============================] - 5s 3ms/step - loss: 1.5244 - accuracy: 0.6584
Epoch 2/30
1641/1641 [==============================] - 5s 3ms/step - loss: 0.5023 - accuracy: 0.8715
Epoch 3/30
1641/1641 [==============================] - 5s 3ms/step - loss: 0.3717 - accuracy: 0.8957
Epoch 4/30
1641/1641 [==============================] - 5s 3ms/step - loss: 0.3242 - accuracy: 0.9081
Epoch 5/30
1641/1641 [==============================] - 5s 3ms/step - loss: 0.2955 - accuracy: 0.9154
Epoch 6/30
1641/1641 [==============================] - 5s 3ms/step - loss: 0.2748 - accuracy: 0.9211
Epoch 7/30
1641/1641 [==============================] - 5s 3ms/step - loss: 0.2575 - accuracy: 0.9265
Epoch 8/30
1641/1641 [==============================] - 5s 3ms/step - loss: 0.2431 - accuracy: 0.9300
Epoch 9/30
1641/1641 [==============================] - 5s 3ms/step - loss: 0.2303 - accuracy: 0.9337
Epoch 10/30
1641/1641 [==============================] - 5s 3ms/step - loss: 0.218

In [53]:
loss, accuracy = model.evaluate(test_data,test_label, batch_size=64,verbose=1)
print("Accuracy: %",format(accuracy * 100))

274/274 [==============================] - 1s 3ms/step - loss: 0.1210 - accuracy: 0.9645
Accuracy: % 96.45143151283264


### 머신러닝의 난제
좋은 데이터, 데이터의 수(충분한가?) 그리고 좋은 알고리즘